In [ ]:
!pip install --upgrade prometheus-api-client urllib3

In [ ]:
from datetime                    import timedelta
from prometheus_api_client       import MetricSnapshotDataFrame, PrometheusConnect
from prometheus_api_client.utils import parse_datetime
from urllib3                     import disable_warnings

disable_warnings()

In [ ]:
PROMETHEUS_URL   = '<PROMETHEUS_URL>'
PROMETHEUS_TOKEN = '<PROMETHEUS_TOKEN>'

In [ ]:
prometheus_connect = PrometheusConnect(
    url         = PROMETHEUS_URL,
    headers     = { 'Authorization' : f'bearer { PROMETHEUS_TOKEN }' },
    disable_ssl = True
)

In [ ]:
prometheus_connect.all_metrics()[:10]

In [ ]:
metric_name = 'pod:container_cpu_usage:sum'
start_time  = parse_datetime('1h')
end_time    = parse_datetime('now')
chunk_size  = timedelta(minutes = 1)

label_config = {
    'prometheus' : 'openshift-monitoring/k8s',
    'namespace'  : '<namespace>'
}

metric_data = prometheus_connect.get_metric_range_data(
    metric_name  = metric_name,
    label_config = label_config,
    start_time   = start_time,
    end_time     = end_time,
    chunk_size   = chunk_size
)

metric_data = MetricSnapshotDataFrame(metric_data)
metric_data

In [ ]:
len(metric_data)